# **YOLOv8 TRAIN**

## **Initial configuration**

**Install packages**

In [ ]:
# !pip install "ultralytics<=8.3.40"
# !pip install ultralytics==8.2.103
# !pip install comet_ml==3.47.4
# !pip install py-cpuinfo
# !pip install psutil
# !pip install gputil

**Import libraries**

In [2]:
# Import libraries
import comet_ml
import cpuinfo
import glob
import GPUtil
import matplotlib.pyplot as plt
import os
import pandas as pd
import platform
import psutil
import shutil
import subprocess
import torch
import urllib.request
import yaml

from PIL import Image
from ultralytics import YOLO

In [ ]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

**Machine specifications**

In [ ]:
def print_machine_specs():
  # Operating System
  print("Operating System:")
  print(f"  Name: {platform.system()} {platform.release()}")
  print(f"  Version: {platform.version()}")
  print(f"  Processor: {platform.processor()}")
  print(f"  Architecture: {platform.architecture()[0]}\n")

  # OS - Distribution
  print("OS - Distribution:")
  print(f"  pretty name: {platform.freedesktop_os_release()['PRETTY_NAME']}")
  print(f"  name: {platform.freedesktop_os_release()['NAME']}")
  print(f"  version: {platform.freedesktop_os_release()['VERSION']}")
  print(f"  version codename: {platform.freedesktop_os_release()['VERSION_CODENAME']}")
  print(f"  id: {platform.freedesktop_os_release()['ID']}")
  print(f"  id like: {platform.freedesktop_os_release()['ID_LIKE']} \n")

  # CPU Information
  cpu = cpuinfo.get_cpu_info()
  print("CPU Information:")
  print(f"  Processor: {cpu['brand_raw']}")
  print(f"  Cores: {psutil.cpu_count(logical=False)}")
  print(f"  Threads: {psutil.cpu_count(logical=True)}")
  print(f"  Max Frequency: {psutil.cpu_freq().max:.2f} MHz\n")

  # Memory Information
  virtual_memory = psutil.virtual_memory()
  print("Memory Information:")
  print(f"  Total: {virtual_memory.total / (1024 ** 3):.2f} GB")
  print(f"  Available: {virtual_memory.available / (1024 ** 3):.2f} GB\n")

  # Disk Information
  disk_usage = psutil.disk_usage('/')
  print("Disk Information:")
  print(f"  Total: {disk_usage.total / (1024 ** 3):.2f} GB")
  print(f"  Used: {disk_usage.used / (1024 ** 3):.2f} GB")
  print(f"  Free: {disk_usage.free / (1024 ** 3):.2f} GB\n")

  # GPU Information (if available)
  try:
    gpus = GPUtil.getGPUs()
    print("GPU Information:")
    for gpu in gpus:
      print(f"  GPU: {gpu.name}")
      print(f"  Memory Total: {gpu.memoryTotal} MB")
      print(f"  Memory Free: {gpu.memoryFree} MB")
      print(f"  Memory Used: {gpu.memoryUsed} MB\n")
  except ImportError:
    print("GPU Information: GPUtil not installed. Install it using `pip install gputil`.\n")

  # Python Information
  print("Python Environment:")
  print(f"  Version: {platform.python_version()}")
  print(f"  Compiler: {platform.python_compiler()}\n")

print_machine_specs()

**Graphic card features**

In [ ]:
# Verify cuda available device
cuda_device_num = torch.cuda.device_count()
if not cuda_device_num:
  print("\033[93m WARNING: No cuda devices found. \033[0m")
else:
  print(f"\033[34m INFO: There {'is' if cuda_device_num == 1 else 'are'} {cuda_device_num} pytorch cuda devices. \033[0m")
  print(f"\033[34m       Pyorch cuda version: {torch.version.cuda} \033[0m")
  print(f"\033[34m-\033[0m" * 35)
  for index in  range(cuda_device_num):
    print(f"\033[34m GPU {index}: {torch.cuda.get_device_name(index)} \033[0m")
    print(f"\033[34m \tTotal cuda device memory {torch.cuda.mem_get_info(0)[1] // 2 ** 20} MiB\033[0m")
    print(f"\033[34m \tFree cuda device memory {torch.cuda.mem_get_info(index)[0] // 2 ** 20} MiB\033[0m\n")

  print("Graphics card information")
  print(subprocess.check_output("nvidia-smi").decode())

**Directory structure**

In [ ]:
# Run local or not
PROJ_ROOT = os.getcwd()
local = True if os.path.basename(PROJ_ROOT) == "notebooks" else False

# Directories path
if not local:
  os.makedirs("models", exist_ok=True)
  data_relative_path ="data"
  models_relative_path = "models"
else:
  data_relative_path ="../data/processed"
  models_relative_path = "../models"

MODELS_DIR = os.path.join(PROJ_ROOT, models_relative_path)
PROCESSED_DATA_DIR = os.path.join(PROJ_ROOT, data_relative_path)

# Print directories paths
print(f"\033[34m INFO: Models in {MODELS_DIR} \033[0m")
print(f"\033[34m INFO: Processed data in {PROCESSED_DATA_DIR} \033[0m")

**Configuration variables**

In [ ]:
# Configuration class
class CFG:
  # Classes
  CLASSES = ["hard_hat",
            "no_hard_hat",
            "no_safety_harness",
            "no_safety_vest",
            "person",
            "safety_harness",
            "safety_vest"]

  # Model
  MODEL_VERSION = "v8"
  MODEL_SIZE = "n" # n, s, m, l, x
  PRETRAINED = True
  MODEL_NAME = f"yolo{MODEL_VERSION}{MODEL_SIZE}"
  BASE_MODEL = f"{MODEL_NAME}.{'pt' if PRETRAINED else 'yaml'}"

  # Dataset
  DATASET_VERSION = 5
  DATASET_NAME = f"ppe_dataset_v{DATASET_VERSION}"
  DATASET_PATH = os.path.join(PROCESSED_DATA_DIR, DATASET_NAME)
  DATASET_YAML_PATH = os.path.join(DATASET_PATH, "data.yaml")

  # Train variables
  DEBUG = False
  EPOCHS = 4 if DEBUG else 4
  PATIENCE = 40
  FRACTION = 0.01 if DEBUG else 0.01
  TRAIN_DEVICE = 0 if cuda_device_num else "cpu"
  TRAIN_MODEL_PATH = os.path.join(MODELS_DIR, "base", BASE_MODEL) if PRETRAINED else BASE_MODEL
  TRAIN_PROJECT_NAME = f"train_{MODEL_NAME}_ppe_detection{'_debug' if DEBUG else ''}"
  TRAIN_PROJECT_PATH = os.path.join(MODELS_DIR, TRAIN_PROJECT_NAME)
  TRAIN_NAME = f"{'pt' if PRETRAINED else 'yaml'}_{EPOCHS}_epochs"

# Print information
if not CFG.DEBUG:
  print("\033[34m INFO: The configuration is ready for TRAINING. \033[0m")
else:
  print("\033[93m WARNING: Configuration is set on DEBUG. \033[0m")

print(f"\nModel information")
print(f"Base modell: {CFG.BASE_MODEL}")

print(f"\nDataset information")
print(f"Dataset path: {CFG.DATASET_PATH}")

print("\nTraining information")
print(f"Train model path: {CFG.TRAIN_MODEL_PATH}")
print(f"Train project path: {CFG.TRAIN_PROJECT_PATH}")
print(f"Train name: {CFG.TRAIN_NAME}")

## **Dataset**

**Verify dataset**

In [ ]:
# Dataset paths
dataset_path_exist = os.path.isdir(CFG.DATASET_PATH)
dataset_yaml_file_exist = os.path.isfile(CFG.DATASET_YAML_PATH)

# Check if path exists
if not dataset_path_exist or not dataset_yaml_file_exist:
  print("\033[93m WARNING: Check dataset path or download again. \033[0m")
else:
  print("\033[34m INFO: The dataset exists. \033[0m\n")
  # Plot dataset yaml file
  with open(CFG.DATASET_YAML_PATH, "r") as file:
    try:
      data = yaml.safe_load(file)
      yaml_data = yaml.dump(data, default_style=False)
      print(yaml_data)

    except yaml.YAMLError as e:
      print("Error reading YAML: ", e)
  file.close()

## **Train**

**Download YOLOv8 base models**

In [ ]:
# Donwload base model function
def donwload_base_model(base_model = "yolov8n.pt"):
  base_models_dir_path = os.path.join(MODELS_DIR, "base")
  os.makedirs(base_models_dir_path, exist_ok=True)

  model_file = os.path.join(base_models_dir_path, base_model)
  # Model from yaml file
  if "yaml" in base_model:
    model_file = base_model
    print(f"Empty {model_file} was charged.")

  # Model from pt file
  else:
    if base_model not in os.listdir(base_models_dir_path):
      print(f"Downloading {base_model} ...")
      try:
        urllib.request.urlretrieve(f"https://github.com/ultralytics/assets/releases/download/v8.2.0/{base_model}", model_file)
        print(f"The model {base_model} was downloaded.")
      except:
        print(f"Something went wrong with downloading ...")
    else:
      print(f"The model {base_model} already exists.")

# Download the base model
donwload_base_model(base_model=CFG.BASE_MODEL)

**Verify the model**

In [ ]:
if ".yaml" != os.path.splitext(CFG.TRAIN_MODEL_PATH)[1]:
  if not os.path.isfile(CFG.TRAIN_MODEL_PATH):
    print(f"\033[31m ERROR: Is posible the base model {CFG.TRAIN_MODEL_PATH} don't exist. \033[0m")
  else:
    print(f"\033[34m INFO: The base model {CFG.TRAIN_MODEL_PATH} exists. \033[0m")
else:
  print(f"\033[34m INFO: The base model {CFG.TRAIN_MODEL_PATH} is a YAML file. \033[0m")

**Train settings**

In [ ]:
# Project configuration
train_path = os.path.join(CFG.TRAIN_PROJECT_PATH, CFG.TRAIN_NAME)
if os.path.isdir(train_path):
  shutil.rmtree(train_path)

print("Train model path:\t", CFG.TRAIN_MODEL_PATH)
print("Train project name:\t", CFG.TRAIN_PROJECT_NAME)
print("Train project path:\t", CFG.TRAIN_PROJECT_PATH)
print("Train name:\t", CFG.TRAIN_NAME)

In [ ]:
cfg_train = {
  "data": CFG.DATASET_YAML_PATH,
  "epochs": CFG.EPOCHS,
  "time": None,
  "patience": CFG.PATIENCE,
  "batch": 16,
  "imgsz": 640,
  "save": True,
  "save_period": -1,
  "cache": False,
  "device": CFG.TRAIN_DEVICE,
  "project": CFG.TRAIN_PROJECT_PATH,
  "name": CFG.TRAIN_NAME,
  "exist_ok": False,
  "pretrained": True,
  "optimizer": "auto",
  "classes": None,
  "multi_scale": False,
  "close_mosaic": 10,
  "resume": False,
  "amp": True,
  "fraction": CFG.FRACTION,
  "profile": False,
  "freeze": None,
  "lr0": 0.01,
  "lrf": 0.01,
  "momentum": 0.937,
  "weight_decay": 0.0005,
  "warmup_epochs": 3.0,
  "warmup_momentum": 0.8,
  "warmup_bias_lr": 0.1,
  "box": 7.5,
  "cls": 0.5,
  "val": True,
  "iou": 0.8,
  "augment": True,
  "plots": True
}

for key, value in cfg_train.items():
    print(key, ":", value)

**Comet ML configuration**

In [ ]:
# Comet config
comet_ml.login()
experiment_config = comet_ml.ExperimentConfig(name=CFG.TRAIN_NAME)
exp = comet_ml.start(project_name=CFG.TRAIN_PROJECT_NAME, experiment_config=experiment_config)

**Load the model**

In [26]:
try:
  train_model = YOLO(CFG.TRAIN_MODEL_PATH)
  print(f"The {CFG.TRAIN_MODEL_PATH} was loaded.")
except:
  print(f"Error loading the {CFG.TRAIN_MODEL_PATH} model.")

**Model information**

In [ ]:
train_model.info()

**Train model**

In [ ]:
train_results = train_model.train(**cfg_train)
exp.end()

**Train results**

In [ ]:
print("map50-95", ":", train_results.box.map)
print("map50", ":", train_results.box.map50)
print("map75", ":", train_results.box.map75)
print("maps", ":", train_results.box.maps)

**Images of the train results**

In [ ]:
train_results_path = train_results.save_dir
train_image_files = [
  i for i in
  glob.glob(f"{train_results_path}/*.png") + glob.glob(f"{train_results_path}/*.jpg")
  if "batch" not in i
]

train_image_files

**Plot images of the train results**

In [ ]:
for image in sorted(train_image_files):
  image_name = image.split("/")[-1]
  print(image_name)

  img = Image.open(image)
  plt.imshow(img)
  plt.axis("off")
  plt.show()

  print("\n")

**Read CSV result**

In [ ]:
df = pd.read_csv(f"{train_results_path}/results.csv")
df = df.rename(columns=lambda x: x.replace(" ", ""))
df

In [ ]:
print("*"*50)
print("\nBest Training Box loss: ", df["train/box_loss"].min(), ", on epoch: ", df["train/box_loss"].argmin() + 1)
print("\nBest Validation Box loss: ", df["val/box_loss"].min(), ", on epoch: ", df["val/box_loss"].argmin() + 1, "\n")

print("="*50)
print("\nBest Training Cls loss: ", df["train/cls_loss"].min(), ", on epoch: ", df["train/cls_loss"].argmin() + 1)
print("\nBest Validation Cls loss: ", df["val/cls_loss"].min(), ", on epoch: ", df["val/cls_loss"].argmin() + 1, "\n")

print("="*50)
print("\nBest Training DFL loss: ", df["train/dfl_loss"].min(), ", on epoch: ", df["train/dfl_loss"].argmin() + 1)
print("\nBest Validation DFL loss: ", df["val/dfl_loss"].min(), ", on epoch: ", df["val/dfl_loss"].argmin() + 1, "\n")

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 20), sharex=True)

#Training and Validation box_loss
ax1.set_title("Box Loss")
ax1.plot(df["epoch"], df["train/box_loss"], label="Training box_loss", marker="o", linestyle="-")
ax1.plot(df["epoch"], df["val/box_loss"], label="Validation box_loss", marker="o", linestyle="-")
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Box Loss")
ax1.legend()
ax1.grid(True)

# Training and Validation cls_loss
ax2.set_title("Cls Loss")
ax2.plot(df["epoch"], df["train/cls_loss"], label="Training cls_loss", marker="o", linestyle="-")
ax2.plot(df["epoch"], df["val/cls_loss"], label="Validation cls_loss", marker="o", linestyle="-")
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Cls loss")
ax2.legend()
ax2.grid(True)

# Training and Validation dfl_loss
ax3.set_title("DFL Loss")
ax3.plot(df["epoch"], df["train/dfl_loss"], label="Training dfl_loss", marker="o", linestyle="-")
ax3.plot(df["epoch"], df["val/dfl_loss"], label="Validation dfl_loss", marker="o", linestyle="-")
ax3.set_xlabel("Epochs")
ax3.set_ylabel("DFL loss")
ax3.legend()
ax3.grid(True)

plt.suptitle("Training Metrics vs. Epochs")
plt.show()

**Validation results**

In [ ]:
train_results_path = train_results.save_dir
valid_image_files = [
  i for i in
  glob.glob(f"{train_results_path}/*.png") + glob.glob(f"{train_results_path}/*.jpg")
  if "val_batch" in i
]

valid_image_files

**Plot images of the validation results**

In [ ]:
for image in sorted(valid_image_files):
  image_name = image.split("/")[-1]
  print(image_name)

  img = Image.open(image)
  plt.imshow(img)
  plt.axis("off")
  plt.show()

  print("\n")